<a href="https://colab.research.google.com/github/karumugamio/SMAIProject-Team31/blob/master/SMT_English_to_Simple_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Opening files from Google Drive and changing current working directory to Data folder 
import os
from google.colab import drive
drive.mount('/gdrive')
os.chdir('/gdrive/My Drive/NLAProjectWS')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
# if lm package import fails please update the NLTK package by !pip install nltk --upgrade to version 3.5
!pip install nltk --upgrade

Requirement already up-to-date: nltk in /usr/local/lib/python3.6/dist-packages (3.5)


In [0]:
# All Import Statements

import nltk
from collections import defaultdict
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams

from nltk.lm import MLE

from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import padded_everygram_pipeline

from nltk.translate.ibm1 import IBMModel1
from nltk.translate.api import AlignedSent

import dill as pickle
import time
import pandas as pd
import numpy as np
import math


In [4]:

os.chdir('/gdrive/My Drive/NLAProjectWS/workspace')
print("NLTK version used in this note book is ",nltk.__version__)
os.listdir()
NormalEngData = '/gdrive/My Drive/NLAProjectWS/Data/v1_wiki.unsimplified'
SimpleEngData = '/gdrive/My Drive/NLAProjectWS/Data/v1_wiki.simple'

SentencePairFile = 'sentence pairs.pickle'
IBMModelFile = 'IBMModel.pickle'
LM_EN_File = 'en_model_trigram.pickle'
LM_SI_File = 'si_model_trigram.pickle'
EN_WORDS_FILE = 'all_en_unique_words.pickle'
SI_WORDS_FILE = 'all_si_unique_words.pickle'

NLTK version used in this note book is  3.5


In [5]:
# uncomment only when you have trouble with nltk resources. This all download everything
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [0]:
allwords_en = open(NormalEngData).read()
allwords_si = open(SimpleEngData).read()
en_words = allwords_en.lower().split()
en_words.append("NULL")
en_words = set (en_words)
si_words = allwords_si.lower().split()
si_words.append("NULL")
si_words = set (si_words)

# Its important to use binary mode 
pklfile = open(EN_WORDS_FILE, 'ab') 
# source, destination 
pickle.dump(en_words, pklfile)                      
pklfile.close() 

pklfile2 = open(SI_WORDS_FILE, 'ab') 
# source, destination 
pickle.dump(si_words, pklfile2)                      
pklfile2.close() 

In [7]:
print("Number of Unique English Words : ",len(en_words))
print("Number of Unique Simple English Words : ",len(si_words))

Number of Unique English Words :  117830
Number of Unique Simple English Words :  107091


## Create Aligned Sentence Pairs

In [0]:
pairs = []
with open(NormalEngData) as normalLines, open(SimpleEngData) as simpleLines: 
    for x, y in zip(normalLines, simpleLines):
        x = x.strip()
        y = y.strip()
        #print("{0}\t{1}".format(x, y))
        pair = (x,y)
        pairs.append(pair)
totalPairs = len(pairs)

## Split Entire Data Corpus into Train Test partition

In [9]:
splitRatio = 0.7
cutoff = round(totalPairs*splitRatio)
print(type(pairs))
train = pairs[0:int(cutoff)]
test = pairs[int(cutoff)+1:int(totalPairs)-1]
print("Length of Total Pairs created is {}".format(totalPairs))
print("Length of train Pairs created is {}".format(len(train)))
print("Length of test Pairs created is {}".format(len(test)))

<class 'list'>
Length of Total Pairs created is 137362
Length of train Pairs created is 96153
Length of test Pairs created is 41207


##Creating Language Models

In [0]:
en_tokenized_text = []
si_tokenized_text = []
for p in pairs:
    en_tokenized_text.append(nltk.word_tokenize(p[0]))
    si_tokenized_text.append(nltk.word_tokenize(p[1]))

In [0]:
#creating placeholder for Trigram Probablity table
trimodel = defaultdict(lambda:defaultdict(lambda:defaultdict(lambda: 0)))#1 for smoothing
#place holder to hold next two words tuple
tri_tuple_model = defaultdict(lambda:defaultdict(lambda: 0))#1 for smoothing

# for en_sentence in train_en_file.lower().split('\n'):
token = nltk.word_tokenize(allwords_en)
for w1, w2, w3 in ngrams(token,3):
    trimodel[w1][w2][w3] += 1;

for w1 in trimodel:
    for w2 in trimodel[w1]:
        sum_values = float(sum(trimodel[w1][w2].values()))
        for w3 in trimodel[w1][w2]:
            trimodel[w1][w2][w3] /= sum_values;
            tri_tuple_model[w3][(w1,w2)] = trimodel[w1][w2][w3] #just for calculation
en_trigram_model = tri_tuple_model


#creating placeholder for Trigram Probablity table
trimodel = defaultdict(lambda:defaultdict(lambda:defaultdict(lambda: 0)))#1 for smoothing
#place holder to hold next two words tuple
tri_tuple_model = defaultdict(lambda:defaultdict(lambda: 0))#1 for smoothing

# for en_sentence in train_en_file.lower().split('\n'):
token = nltk.word_tokenize(allwords_si)
for w1, w2, w3 in ngrams(token,3):
    trimodel[w1][w2][w3] += 1;

for w1 in trimodel:
    for w2 in trimodel[w1]:
        sum_values = float(sum(trimodel[w1][w2].values()))
        for w3 in trimodel[w1][w2]:
            trimodel[w1][w2][w3] /= sum_values;
            tri_tuple_model[w3][(w1,w2)] = trimodel[w1][w2][w3] #just for calculation
si_trigram_model = tri_tuple_model

In [0]:
# Simplifying read and preprocess for creating models
def read_sents(filename):
    sents = []
    c=0
    with open(filename,'r') as fi:
        for li in fi:
            sents.append(li.split())
    return sents

In [13]:
# Arrived Max Count as 96153 by 0.7 times the available corpus.
# We are using IBM Model 1 for creating Translation Probablity
max_count = 96153
eng_sents_all = read_sents(NormalEngData)
fr_sents_all = read_sents(SimpleEngData)
eng_sents = eng_sents_all[:max_count]
fr_sents = fr_sents_all[:max_count]
print("Size of english sentences: ", len(eng_sents))
print("Size of french sentences: ", len(fr_sents))
aligned_text = []
for i in range(len(eng_sents)):
    al_sent = AlignedSent(fr_sents[i],eng_sents[i])
    aligned_text.append(al_sent)
print("Training IBM model 1")
start = time.time()
ibm_model = IBMModel1(aligned_text,5)

end = time.time()
executionTime = end - start
print("Training complete, Time taken for training model is :",executionTime)

Size of english sentences:  96153
Size of french sentences:  96153
Training IBM model 1
Training complete, Time taken for training model is : 478.0055296421051


In [0]:
# Its important to use binary mode 
dbfile = open(IBMModelFile, 'ab') 
# source, destination 
pickle.dump(ibm_model, dbfile)                      
dbfile.close() 

In [0]:
#si_model
# Its important to use binary mode 
dbfile1 = open(LM_SI_File, 'ab') 
# source, destination 
pickle.dump(si_trigram_model, dbfile1)                      
dbfile1.close() 

In [0]:
#si_model
# Its important to use binary mode 
dbfile2 = open(LM_EN_File, 'ab') 
# source, destination 
pickle.dump(en_trigram_model, dbfile2)                      
dbfile2.close() 

In [0]:
# Its important to use binary mode 
dbfile3 = open(SentencePairFile, 'ab') 
# source, destination 
pickle.dump(pairs, dbfile3)                      
dbfile3.close() 

In [18]:
#Loading all Pickle Files for Creating actual SMT using HMM- Viterbi Algorithm
# Source Tutorial for Viterbi- https://youtu.be/ECu_KQV3V30
'''
pickle_in = open(SentencePairFile,"rb")
pairs = pickle.load(pickle_in)
pickle_in = open(LM_EN_File,"rb")
en_model = pickle.load(pickle_in)
pickle_in = open(LM_SI_File,"rb")
si_model = pickle.load(pickle_in)
pickle_in = open(IBMModelFile,"rb")
ibm_model = pickle.load(pickle_in)

pickle_in = open(EN_WORDS_FILE,"rb")
en_words = pickle.load(pickle_in)

pickle_in = open(SI_WORDS_FILE,"rb")
si_words = pickle.load(pickle_in)

print("Number of Unique English Words : ",len(en_words))
print("Number of Unique Simple English Words : ",len(si_words))
'''

'\npickle_in = open(SentencePairFile,"rb")\npairs = pickle.load(pickle_in)\npickle_in = open(LM_EN_File,"rb")\nen_model = pickle.load(pickle_in)\npickle_in = open(LM_SI_File,"rb")\nsi_model = pickle.load(pickle_in)\npickle_in = open(IBMModelFile,"rb")\nibm_model = pickle.load(pickle_in)\n\npickle_in = open(EN_WORDS_FILE,"rb")\nen_words = pickle.load(pickle_in)\n\npickle_in = open(SI_WORDS_FILE,"rb")\nsi_words = pickle.load(pickle_in)\n\nprint("Number of Unique English Words : ",len(en_words))\nprint("Number of Unique Simple English Words : ",len(si_words))\n'

In [0]:
def translate(inputSentence):
  inputSentence = inputSentence.split()

  pi = defaultdict(lambda:float(1))
  bp = defaultdict(lambda:" ")

  for k in range(0,len(inputSentence)):
    for eachword in si_words:
      for(w1,w2) in si_trigram_model[eachword] :
        pi_now = pi[k-1]*si_trigram_model[eachword][(w1,w2)] * ibm_model.translation_table[inputSentence[k]][eachword]
        if (pi[k]==1 or pi[k]<pi_now):
          pi[k] = pi_now
          bp[k+1] = eachword
    
    predicted_text = ''
    for i in range (len(bp)):
      predicted_text = predicted_text+' '+bp[i]
  return predicted_text

In [87]:
EN_test = "Soria made seven appearances for the Peru national football team during 2000"
translate(EN_test)

'   cat made seven appearances for the capitals national football team during 2000'

In [103]:
len(test)
import pandas as pd
df = pd.DataFrame(columns=['Source', 'Target', 'Predicted'])
for i in range(100):
  print("Translating Sentence pair #:",i)
  predicted_result = translate(test[i][0])
  df.loc[i]=[test[i][0],test[i][1],predicted_result]

Translating Sentence pair #: 0
Translating Sentence pair #: 1
Translating Sentence pair #: 2
Translating Sentence pair #: 3
Translating Sentence pair #: 4
Translating Sentence pair #: 5
Translating Sentence pair #: 6
Translating Sentence pair #: 7
Translating Sentence pair #: 8
Translating Sentence pair #: 9
Translating Sentence pair #: 10
Translating Sentence pair #: 11
Translating Sentence pair #: 12
Translating Sentence pair #: 13
Translating Sentence pair #: 14
Translating Sentence pair #: 15
Translating Sentence pair #: 16
Translating Sentence pair #: 17
Translating Sentence pair #: 18
Translating Sentence pair #: 19
Translating Sentence pair #: 20
Translating Sentence pair #: 21
Translating Sentence pair #: 22
Translating Sentence pair #: 23
Translating Sentence pair #: 24
Translating Sentence pair #: 25
Translating Sentence pair #: 26
Translating Sentence pair #: 27
Translating Sentence pair #: 28
Translating Sentence pair #: 29
Translating Sentence pair #: 30
Translating Senten

In [102]:
pd.__version__

'1.0.3'

In [0]:
df.to_pickle('SMT 100 Sentence pd.pkl')
df.to_csv('SMT 100 Sentence pd.csv')

#end of the model


Please refer CSv file in google Drive work space for results. 
